# ch 16_8 SARIMAX

지금까지 배운 ARIMA, SARIMA 모델 등은 시간에 흐름에 따른 종속 변수의 변화만 고려했습니다. 여기에 exogenous variable, 외생 변수의 영향을 포함시킨 것이 SARIMAX 모델입니다.

예를들어 치킨집의 매출 시계열 데이터가 주어졌다고 가정하겠습니다. 국가대표 스포츠 경기가 있는 날엔 치킨집의 매출이 급증합니다만, 시간만 고려한 모델은 이를 반영하지 못합니다. 이런 외생 변수를 시계열 모델에 포함시킬 수 있는 모델이 SARIMAX입니다.

## 데이터 셋 준비

데이터 셋 링크: https://www.kaggle.com/datasets/prakharprasad/time-series-data-1?select=RestaurantVisitors.csv  
참고 자료: https://www.kaggle.com/code/prakharprasad/restaurant-visitors-forecast-with-sarimax/log

### 계절성 확인

timeseries decompose를 이용해서 데이터가 계절성을 보이는지 확인해보겠습니다.

확대해서 그래프를 그려보면 일주일 간격으로 계절성을 보이는 것을 알 수 있습니다.

## SARIMA 

### 모델 학습
외생 변수의 영향을 파악하기 위해서 먼저 SARIMA 모델을 학습시키고, 예측 결과를 시각화 해보겠습니다. auto_arima와 train set으로 모델을 학습시키겠습니다.

### 인퍼런스

test set에 대해서 예측을 내려보겠습니다.

### 결과 시각화

공휴일을 함께 시각화해보면 공휴일에 SARIMA 모델이 예측한 값보다 실제 방문객이 많음을 알 수 있습니다. 즉, 공휴일이라는 외생 변수를 모델에 추가할 경우, 성능 향상을 기대할 수 있습니다.

## SARIMAX 모델 학습

SARIMAX 모델은 SARIMA와 달리 auto_arima로 바로 학습시킬 수가 없습니다. 때문에 먼저 auto_arima로 적합한 파라미터를 찾고, 이를 이용하여 statsmodels에 내장된 SARIMAX 모델을 학습시켜야 합니다. 외생 변수의 영향을 모델에 포함시키기 위해서 exogeneous 옵션에 holiday를 넣어주겠습니다.

### auto_arima를 이용한 파라미터 탐색

### SARIMAX 모델 학습

train set과 auto_arima를 이용해 탐색한 파라미터를 가지고 SARIMAX 모델을 학습시킵니다. 마찬가지로 exog에 holiday를 넣어줍니다.

### 인퍼런스

### 결과 시각화

SARIMA 모델과 비교를 해보면 공휴일에 실제 방문객과 더 근접하게 예측값을 내려주는 것을 확인할 수 있습니다.

## 메트릭 비교

SARIMA 모델과 SARIMAX 모델의 RMSE를 비교해보면 train set에서는 SARIMAX의 결과가 더 좋지만, test set에서는 SARIMA 모델이 조금 더 좋습니다. 이는 아마도 test set에 포함된 공휴일의 수가 적어서 발생한 현상으로 추측됩니다.

## 정리

이번 챕터에서는 외생 변수의 영향을 함께 고려할 수 있는 SARIMAX 모델에 대해서 배워봤습니다. 그리고 대표적인 외생 변수인 공휴일 여부를 포함하여 SARIMAX 모델을 학습시켰고, SARIMA 모델과 성능을 비교해보았습니다.